<a href="https://colab.research.google.com/github/rickiepark/fine-tuning-llm/blob/main/Chapter0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0장 LLM 미세 튜닝 레시피

사용된 패키지
* torch 2.9.0
* transformers 5.2.0
* datasets 4.0.0
* bitsandbytes 0.49.1
* trl 0.26.2
* peft 1.18.0
* accelerate 1.12.0

### 스포일러

이 장에서는 바로 본론으로 들어가 소규모 언어 모델인 마이크로소프트의 Phi-3-mini-4k-instruct 모델을 미세 튜닝하여 영어를 요다체(Yoda-speak)로 바꾸어 보겠습니다. 이 첫 번째 장을 그냥 따라하는 일종의 레시피로 생각할 수 있습니다. "일단 해보고 궁금한 건 나중에 알아보자" 식의 장이죠.

이 장에서는 다음과 같은 내용을 배웁니다.

- BitsAndBytes를 사용해 양자화된 모델(quantized model)을 로드합니다.
- 허깅 페이스(Hugging Face)의 peft를 사용해 LoRA(low-rank adapter)를 설정합니다.
- 데이터셋을 로드하고 포맷팅합니다.
- 허깅 페이스의 trl에서 제공하는 SFTTrainer 클래스를 사용해 모델을 지도 학습 미세 튜닝(supervised fine-tuning)합니다.
- 미세 튜닝된 모델을 사용해 요다체 문장을 생성합니다.

### 설정

훈련 재현성을 위해 책에서 사용하는 버전과 동일한 버전을 설치합니다:

In [1]:
!pip install bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 40.8 MB/s eta 0:00:00


### 라이브러리 임포트

In [2]:
import os
import torch
from contextlib import nullcontext
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer

## 양자화된 베이스 모델 로드하기

먼저 GPU RAM을 덜 차지하는 양자화된 모델을 로드합니다. 양자화된 모델은 원본 가중치를 더 적은 비트로 표현된 근삿값으로 바꾼 것입니다. 모델을 양자화하는 가장 간단한 방법은 가중치를 32비트 부동 소수점(FP32) 숫자에서 4비트 부동 소수점(NF4)로 바꾸는 것입니다. 간단하지만 이런 강력한 변경이 모델의 메모리 사용량을 약 8배나 줄입니다.

`from_pretrained()` 메서드를 사용해 모델을 로드할 때 `quantization_config` 매개변수에 `BitsAndBytesConfig` 객체를 전달합니다. 여러 모델을 자유롭게 테스트할 수 있도록 허깅 페이스의 `AutoModelForCausalLM`를 사용하겠습니다. 이 클래스를 허깅 페이스에 있는 어떤 저장소를 선택하는지에 따라 로드되는 모델이 결정됩니다.

양자화된 모델을 로드하는 코드는 다음과 같습니다:

In [3]:
bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float32
)
repo_id = 'microsoft/Phi-3-mini-4k-instruct'
model = AutoModelForCausalLM.from_pretrained(repo_id,
                                             device_map="cuda:0",
                                             quantization_config=bnb_config
)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

<blockquote class="note">
  <p>
    <em>"Phi-3-Mini-4K-Instruct는 38억 개의 파라미터를 가지고 있으며 Phi-3 데이터셋에서 훈련된 경량의 최신 오픈 소스 모델입니다. 이 데이터셋은 합성 데이터와 필터링된 공개 웹사이트의 데이터로 구성되며 고품질의 추론 중심 데이터입니다. 이 모델은 Phi-3 제품군에 속합니다. Mini 모델은 4K와 128K 문맥 길이(토큰 수)를 가진 버전이 있습니다."</em>
    <br>
    출처: <a href="https://huggingface.co/microsoft/Phi-3-mini-4k-instruct">허깅 페이스</a>
  </p>
</blockquote>

모델이 로드된 후 `get_memory_footprint()` 메서드를 사용해 모델이 얼마만큼의 메모리를 차지하고 있는지 확인할 수 있습니다.

In [4]:
print(model.get_memory_footprint()/1e6)

2206.341312


양자화되었더라도 모델은 2기가바이트 이상의 RAM을 차지합니다. 양자화 과정은 주로 **트랜스포머 디코더 블록**(Transformer decoder block)(종종 층이라고두 부릅니다)에 있는 **선형 층**(linear layer)을 대상으로 합니다:

In [5]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLUActivation()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=

**양자화된 모델**(quantized model)은 바로 추론에 사용할 수 있지만 추가적으로 더 훈련할 수는 없습니다. 양자화의 핵심인 Linear4bit 층이 메모리를 적게 차지하지만 업데이트할 수 없기 때문입니다.

대신 어댑터를 추가하여 모델의 동작을 바꿀 수 있습니다.

## LoRA 설정하기

LoRA(Low-Rank Adapter) 어댑터를 양자화된 각 층에 추가할 수 있습니다. **어댑터**(adapter)는 일반적인 Linear 층과 거의 비슷하며 쉽게 업데이트할 수 있습니다. 여기서 기발한 점은 이 어댑터가 양자화된 층보다 훨씬 작다는 것입니다.

양자화된 층이 동결(frozen)되었기 때문에 (즉, 업데이트되지 않으므로) 양자화된 모델에 LoRA 어댑터를 추가했을 때 훈련 가능한 파라미터의 전체 개수가 원래의 1% (또는 그 미만으로) 크게 줄어 듭니다.

LoRA 어댑터를 설정하는 단계는 세 개입니다.

* 훈련 과정에서 수치 안정성을 향상시키기 위해 `prepare_model_for_kbit_training()`를 호출합니다.
* `LoraConfig` 인스턴스를 만듭니다.
* `get_peft_model()` 메서드를 사용해 양자화된 베이스 모델(base model)에 이 설정을 적용합니다.

이 단계를 앞서 로드한 모델에 적용해 보죠:

In [6]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,                   # 어댑터의 랭크(rank)가 작을수록 훈련할 파라미터가 적습니다.
    lora_alpha=16,         # 일반적으로 2*r
    bias="none",           # 주의: 편향을 훈련하면 베이스 모델의 동작이 바뀔 수 있습니다.
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    # 이 글을 쓰는 시점에 Phi-3와 같은 최신 모델은
    # 대상 모듈을 수동으로 지정해야 합니다.
    target_modules=['o_proj', 'qkv_proj', 'gate_up_proj', 'down_proj'],
)

model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (

다른 LoRA 층(`qkv_proj`, `gate_up_proj`, `down_proj`)은 출력 길이를 짧게하기 위해 간단하게 나타냈습니다.

<blockquote class="warning">
  <p>
    혹시 다음과 같은 오류가 발생했나요?
    <br>
    <br>
    <tt>ValueError: Please specify `target_modules` in `peft_config`</tt>
    <br>
    <br>
    유명한 모델을 사용한다면 대부분 target_modules를 지정할 필요가 없습니다. peft 라이브러리가 자동으로 적절한 대상을 선택합니다. 하지만 인기있는 모델이 출시되는 시점과 라이브러리가 업데이트되는 시점 사이에는 간격이 있을 수 있습니다. 따라서 위와 같은 오류를 만났다면 모델에 있는 양자화된 층을 찾아 target_modules 매개변수에 해당 층의 이름을 입력하세요.
  </p>
</blockquote>

양자화된 층(`Linear4bit`)이 `lora.Linear4bit`로 바뀌었습니다. 이 층은 양자화된 층인 `base_layer`와 일반적인 `Linear` 층(`lora_A`와 `lora_B`)을 섞은 것입니다.

추가된 층은 모델의 크기를 조금만 더 증가시킵니다. 하지만 prepare_model_for_kbit_training() 함수가 양자화되지 않은 다른 모든 층을 **단정밀도**(single precision)(**FP32**)로 바꿉니다. 이로 인해 모델이 20% 더 커집니다:

In [7]:
print(model.get_memory_footprint()/1e6)

2651.074752


대부분의 파라미터가 동결되었기 때문에 전체 파라미터 중에서 적은 부분만 훈련됩니다. LoRA 만세!

In [8]:
trainable_parms, tot_parms = model.get_nb_trainable_parameters()
print(f'훈련 가능한 파라미터:             {trainable_parms/1e6:.2f}M')
print(f'총 파라미터:                 {tot_parms/1e6:.2f}M')
print(f'훈련 가능한 파라미터의 비율: {100*trainable_parms/tot_parms:.2f}%')

훈련 가능한 파라미터:             12.58M
총 파라미터:                 3833.66M
훈련 가능한 파라미터의 비율: 0.33%


모델을 미세 튜닝할 준비를 마쳤지만 한 가지 중요한 요소인 데이터셋이 아직 준비되지 않았습니다.

## 데이터셋 포맷팅하기

<blockquote style="quotes: none !important;">
  <p>
    <em>"요다 처럼, 말해라, 반드시. 흐음."</em>
    <br>
    <br>
    마스터 요다
  </p>
</blockquote>

[`yoda_sentences`](https://huggingface.co/datasets/dvgodoy/yoda_sentences) 데이터셋은 영어를 요다체로 바꾼 720개 문장으로 구성되어 있습니다. 이 데이터셋은 허깅 페이스 허브(Hugging Face Hub)에서 다운받을 수 있으며 `datasets` 라이브러리의 load_dataset() 메서드로 손쉽게 로드할 수 있습니다:

In [9]:
dataset = load_dataset("dvgodoy/yoda_sentences", split="train")
dataset

README.md:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentences.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/720 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'translation', 'translation_extra'],
    num_rows: 720
})

이 데이터셋에는 세 개의 열이 있습니다:

* 원본 영어 문장 (`sentence`)
* 요다체로 바꾼 문장(`translation`)
* Yesss와 Hrrmm 감탄사를 포함한 향상된 번역(`translation_extra`)

In [10]:
dataset[0]

{'sentence': 'The birch canoe slid on the smooth planks.',
 'translation': 'On the smooth planks, the birch canoe slid.',
 'translation_extra': 'On the smooth planks, the birch canoe slid. Yes, hrrrm.'}

모델을 미세 튜팅하기 위해 사용할 `SFTTrainer` 클래스가 자동으로 데이터셋을 **대화 포맷**(conversational format)으로 처리할 수 있습니다.

```
{"messages":[
  {"role": "system", "content": "<general directives>"},
  {"role": "user", "content": "<prompt text>"},
  {"role": "assistant", "content": "<ideal generated text>"}
]}
```

이전 버전에서는 `prompt`와 `completion` 열만 두 개만 있으면 데이터셋이 자동으로 대화 포맷으로 변환되었습니다. 이제는 더이상 이 방식이 지원되지 않으므로 직접 변환하는 것이 좋습니다. 다음에 나오는 `format_dataset()` 함수는 `trl` 패키지를 참고하여 작성한 것입니다.

In [11]:
# trl.extras.dataset_formatting.instructions_formatting_function을 참고함.
# 데이터셋을 (더이상 지원되지 않는) prompt/completion 포맷에서 대화 포맷으로 변경합니다.
def format_dataset(examples):
    if isinstance(examples["prompt"], list):
        output_texts = []
        for i in range(len(examples["prompt"])):
            converted_sample = [
                {"role": "user", "content": examples["prompt"][i]},
                {"role": "assistant", "content": examples["completion"][i]},
            ]
            output_texts.append(converted_sample)
        return {'messages': output_texts}
    else:
        converted_sample = [
            {"role": "user", "content": examples["prompt"]},
            {"role": "assistant", "content": examples["completion"]},
        ]
        return {'messages': converted_sample}

이 함수는 `prompt`와 `completion` 열을 가진 샘플을 기대하므로 데이터셋에 적용하기 전에 열 이름을 바꾸어야 합니다.

In [12]:
dataset = dataset.rename_column("sentence", "prompt")
dataset = dataset.rename_column("translation_extra", "completion")
dataset = dataset.map(format_dataset)
dataset = dataset.remove_columns(["prompt", "completion", "translation"])
messages = dataset[0]['messages']
messages

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

[{'content': 'The birch canoe slid on the smooth planks.', 'role': 'user'},
 {'content': 'On the smooth planks, the birch canoe slid. Yes, hrrrm.',
  'role': 'assistant'}]

### 토크나이저

실제 훈련으로 넘어가기 전에 이 모델에 해당하는 **토크나이저**(tokenizer)를 로드해야 합니다. 토크나이저는 전체 과정에서 중요한 역할을 담당하며, 모델을 훈련할 때와 동일한 방식으로 텍스트를 토큰(token)으로 바꾸어 줍니다.

지시와 채팅 모델의 경우 토크나이저에는 **채팅 템플릿**(chat template)도 담겨 있으며, 이를 통해 다음과 같은 내용을 알 수 있습니다.

- 사용할 **특수 토큰**과 위치
- 시스템 지시 사항, **사용자 프롬프트**(prompt), 모델 응답의 위치
- **생성 프롬프트**, 즉 모델의 응답을 트리거(trigger)하는 특수 토큰("모델에게 질의하기" 절에서 자세히 다룹니다)

**<중요>**
EOS 토큰을 고유하게 유지하는 것이 중요합니다. Phi-3 같은 일부 모델에서는 EOS 토큰이 PAD 토큰으로도 사용됩니다. 이로 인해 훈련 중에 EOS 토큰이 마스킹되어 토큰 생성이 끊임없이 이어질 수 있습니다. 이 문제를 피하기 위해 UNK 토큰을 PAD 토큰으로 할당합니다.

In [13]:
tokenizer = AutoTokenizer.from_pretrained(repo_id)
# EOS 토큰을 고유하게 만들기 위해 UNK 토큰을 PAD 토큰으로 할당합니다.
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

tokenizer.chat_template

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

"{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'user' %}{{'<|user|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>\n' + message['content'] + '<|end|>\n'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% else %}{{ eos_token }}{% endif %}"

복잡해 보이는 템플릿은 신경쓰지 마세요(보기 좋도록 줄바꿈과 들여쓰기를 추가했습니다). 토크나이저는 다음처럼 메시지를 적절한 태그를 사용해 일관성 있는 블록으로 구성합니다(`tokenize=False`로 지정하면 숫자 토큰 ID의 시퀀스가 아니라 텍스트가 반환됩니다):

In [14]:
print(tokenizer.apply_chat_template(messages, tokenize=False))

<|user|>
The birch canoe slid on the smooth planks.<|end|>
<|assistant|>
On the smooth planks, the birch canoe slid. Yes, hrrrm.<|end|>
<|endoftext|>



각 대화는 `<|user|>` 또는 `<|assistant|>`로 시작하고 `<|end|>`로 끝납니다. 또한 `<|endoftext|>`는 전체 블록의 끝을 나타냅니다.

모델이 다르면 템플릿이 다르고, 문장과 블록의 시작과 끝을 나타내는 토큰이 다를 수 있습니다.

이제 미세 튜닝을 수행할 준비를 마쳤습니다!

## SFTTrainer를 사용해 미세 튜닝하기

모델의 규모에 상관없이 미세 튜닝은 모델을 처음부터 훈련하는 것과 정확히 동일한 훈련 과정을 거칩니다. 파이토치로 훈련 루프를 직접 구현하거나 허깅 페이스의 `Trainer` 클래스를 사용해 모델을 미세 튜닝할 수 있습니다.

하지만 (`Trainer`를 상속한) `SFTTrainer`를 사용하는 것이 훨씬 쉽습니다. 다음의 네 가지 매개변수 값을 제공하기만 하면 대부분의 세부 사항을 처리해 주기 때문입니다.

* 모델
* 토크나이저
* 데이터셋
* 설정 객체

처음 세 개는 이미 준비되었으므로 마지막 설정 객체를 만들어 보죠.

### SFTConfig

설정 객체에는 많은 매개변수가 있습니다. 이를 네 개의 그룹으로 나누어 보죠:

* 메모리 사용 최적화 매개변수는 **그레이디언트 누적**(gradient accumulation) 및 **그레이디언트 체크포인팅**(gradient checkpointing)과 관련이 있습니다.
* `max_seq_length`와 같은 데이터셋 관련 매개변수와 시퀀스 패킹(packing) 여부
* `learning_rate` 및 `num_train_epochs` 같은 일반적인 훈련 매개변수
* `output_dir`(모델을 훈련한 다음 허깅 페이스 허브에 저장할 경우 모델의 이름으로 사용됩니다), `logging_dir`, `logging_steps`와 같은 환경 및 로깅 매개변수

**학습률**(learning rate)이 매우 중요한 매개변수입니다(처음에는 베이스 모델 훈련에 사용한 학습률을 시도해볼 수 있습니다). 하지만 실제로는 **최대 시퀀스 길이**(maximum sequence length)가 메모리 부족 문제를 일으킬 가능성이 더 높습니다.

주어진 문제에서 가능한 가장 짧은 max_seq_length를 선택하세요. 여기에서는 영어와 요다체 문장 모두 매우 짧습니다. 64개 토큰이면 프롬프트, 텍스트 완성, 특수 토큰을 포함하기에 충분합니다.

<blockquote class="tip">
  <p>
    나중에 보게 되겠지만 플래시 어텐션(flash attention)을 사용하면 메모리 부족 문제를 피하면서 더 긴 시퀀스를 사용할 수 있습니다.
  </p>
</blockquote>

****
**중요 업데이트**: `trl` 버전 0.20에서 `SFTConfig`에 몇 가지 중요한 변경 사항이 있습니다:
- `packing_strategy='wrapped'`로 지정하지 않으면 패킹이 이전과 다르게 수행됩니다.
- `max_seq_length` 매개변수가 `max_length`로 이름이 바뀌었습니다.
- `bf16` 매개변수 기본값이 `True`이지만 이 글을 쓰는 시점(2025년 10월)에 실제로 BF16 타입이 가능한지 여부를 확인하지 앟습니다. 따라서 설정에 이 매개변수를 명시적으로 포함시킵니다.
****

In [15]:
sft_config = SFTConfig(
    ## 그룹 1: 메모리 사용
    # 이 매개변수들은 GPU RAM을 최대한 활용하도록 돕습니다.
    # 체크포인팅
    gradient_checkpointing=True,   # 메모리가 많이 절약됩니다.
    # 파이토치 새 버전에서 예외를 피하기 위해 지정합니다.
    gradient_checkpointing_kwargs={'use_reentrant': False},
    # 그레이디언트 누적과 배치 크기
    # (업데이트를 위한) 실제 배치 크기는 마이크로 배치 크기와 같습니다.
    gradient_accumulation_steps=1,
    # 초기 (마이크로) 배치 크기
    per_device_train_batch_size=16,
    # 배치 크기가 메모리 부족을 일으키면 문제가 해결될 때까지 반으로 나눕니다.
    auto_find_batch_size=True,

    ## 그룹 2: 데이터셋 관련
    max_length=64,
    # 데이터셋 패킹을 한다는 것은 패딩이 필요 없다는 의미입니다.
    packing=True,
    packing_strategy='wrapped',

    ## 그룹 3: 일반적인 훈련 매개변수
    num_train_epochs=10,
    learning_rate=3e-4,
    # 8-비트 Adam 옵티마이저 - LoRA를 사용하는 경우 큰 도움이 되지 않습니다!
    optim='paged_adamw_8bit',

    ## 그룹 4: 로깅 매개변수
    logging_steps=10,
    logging_dir='./logs',
    output_dir='./phi3-mini-yoda-adapter',
    report_to='none',

    ## 그외
    # 가능한 경우에만 bf16으로 훈련하도록
    bf16=torch.cuda.is_bf16_supported(including_emulation=False)
)

**<중요>** 이 글을 쓰는 시점에 `trl`의 버전(0.21)은 지원 환경 여부에 상관없이 기본적으로 bf16 데이터 타입을 사용해 혼합 정밀도 훈련을 수행합니다. 문제를 방지하기 위해 GPU 지원 여부에 따라 `bf16` 매개변수를 설정합니다.

### `SFTTrainer`

<blockquote style="quotes: none !important;">
  <p>
    <em>"이제 훈련 시간이다!"</em>
    <br>
    <br>
    헐크
  </p>
</blockquote>

****
**중요 업데이트**: `trl` 0.23 버전짜기 LoRA 설정이 모델에 이미 적용되어 있으면 훈련이 실패하는 문제가 있었습니다. `trainer` 객체가 어댑터를 포함해 전체 모델을 동결하기 때문입니다.

이 예제에서처럼 모델이 어댑터를 포함하고 있다면 원본 모델(`model.base_model.model`)과 `peft_config` 매개변수를 함께 사용해야 훈련이 됩니다.

이 문제는 2025년 10월 릴리스된 `trl` 0.23.1 버전에서 수정되었습니다.
****

드디어 지도 학습 미세 튜닝을 위한 훈련 객체를 만들 수 있습니다:

In [16]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=dataset,
)

Tokenizing train dataset:   0%|          | 0/720 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/720 [00:00<?, ? examples/s]

`SFTTrainer`에 사전에 전처리된 데이터셋을 전달했으므로 미니 배치가 어떻게 구성되었는지 확인해 볼 수 있습니다:

In [17]:
dl = trainer.get_train_dataloader()
batch = next(iter(dl))

레이블(label)을 확인해 보죠. 무엇보다도 앞에서 직접 레이블을 지정하지 않았습니다. 그렇죠?

In [18]:
batch['input_ids'][0], batch['labels'][0]

(tensor([ 3974, 29892,  4337,   278,   325,   271, 29892,   366,  1818, 29889,
         32007, 32000, 32010,   450,   289,   935,   310,   278,   282,   457,
          5447,   471,   528,  4901,   322,  6501, 29889, 32007, 32001, 26399,
          1758,  4317, 29889,  1383,  4901,   322,  6501, 29892,   278,   289,
           935,   310,   278,   282,   457,  5447,   471, 29889, 32007, 32000,
         32010,   951,  5989,  2507, 17354,   322, 13328,   297,   278,  6416,
         29889, 32007, 32001,   512], device='cuda:0'),
 tensor([ 3974, 29892,  4337,   278,   325,   271, 29892,   366,  1818, 29889,
         32007, 32000, 32010,   450,   289,   935,   310,   278,   282,   457,
          5447,   471,   528,  4901,   322,  6501, 29889, 32007, 32001, 26399,
          1758,  4317, 29889,  1383,  4901,   322,  6501, 29892,   278,   289,
           935,   310,   278,   282,   457,  5447,   471, 29889, 32007, 32000,
         32010,   951,  5989,  2507, 17354,   322, 13328,   297,   278,  64

입력과 정확하게 동일한 값으로 레이블이 자동으로 추가되었습니다. 따라서 이는 **자기 지도 학습 미세 튜닝**(self-supervised fine-tuning)입니다.

레이블을 하나씩 이동시키는 것도 자동으로 처리되기 때문에 이에 대해 신경쓸 필요가 없습니다.

<blockquote class="note">
  <p>
    38억 개의 파라미터를 가진 모델이지만 앞의 설정을 사용하면 6GB RAM의 GTX 1060 같은 개인용 GPU에서 8개의 미니 배치로 훈련을 수행할 수 있습니다. 정말이에요!
    <br>
    이 경우 훈련을 완료하는데 약 35분이 걸립니다.
  </p>
</blockquote>

그다음 `train()` 메서드를 호출합니다:

In [19]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.


Step,Training Loss
10,2.850900
20,1.826200
30,1.603900
40,1.520800
50,1.399600
60,1.294500
70,1.191500
80,0.989900
90,0.901700
100,0.630000


TrainOutput(global_step=220, training_loss=0.8352644606070085, metrics={'train_runtime': 110.7509, 'train_samples_per_second': 30.79, 'train_steps_per_second': 1.986, 'total_flos': 4890970340720640.0, 'train_loss': 0.8352644606070085, 'epoch': 10.0})

## 모델에게 질의하기

이제 모델에게 짧은 문장을 제공하면 요다 말투의 문장이 생성되어야 합니다.

이 모델은 적절하게 포맷팅된 입력이 필요합니다. (이 경우 `user`에 해당하는) 메시지 리스트를 만들고 모델이 대답할 차례라는 것을 알려 주어야 합니다.

이것이 `add_generation_prompt` 매개변수의 목적입니다. 대화의 끝에 `<|assistant|>`를 추가하여 모델이 다음 단어를 예측하게 하고 `<|endoftext|>` 토큰이 예측될 때까지 계속 진행합니다.

다음 헬퍼 함수는 (대화 포맷으로) 메시지를 만들고 채팅 템플릿을 적용한 후 마지막에 특수 토큰 `<|assistant|>`를 추가합니다.

In [20]:
def gen_prompt(tokenizer, sentence):
    converted_sample = [
        {"role": "user", "content": sentence},
    ]
    prompt = tokenizer.apply_chat_template(converted_sample,
                                           tokenize=False,
                                           add_generation_prompt=True)
    return prompt

샘플 문장으로 프폼프트를 생성해 보죠:

In [21]:
sentence = 'The Force is strong in you!'
prompt = gen_prompt(tokenizer, sentence)
print(prompt)

<|user|>
The Force is strong in you!<|end|>
<|assistant|>



프롬프트가 제대로 만들어진 것 같습니다. 이를 사용해 응답을 생성해 보죠. `generate()` 함수는 다음과 같은 작업을 합니다:

* 프롬프트를 토큰화하여 토큰 ID의 텐서를 만듭니다(채팅 템플릿으로 특수 토큰을 이미 추가했으므로 `add_special_tokens`를 `False`로 지정합니다).
* 모델을 **평가 모드**(`evaluation mode`)로 설정합니다.
* 모델의 `generate()` 메서드를 호출하여 출력(토큰 ID)을 생성합니다.
  * 모델이 혼합 정밀도로 훈련되었다면 데이터 타입 간의 변환을 자동으로 처리하기 위해 `autocast()` 컨텍스트 관리자로 생성 과정을 감쌉니다.
* 생성된 토큰 ID를 텍스트로 디코딩합니다.

In [22]:
def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=False):
    tokenized_input = tokenizer(prompt, add_special_tokens=False,
                                return_tensors="pt").to(model.device)

    print(tokenized_input)
    model.eval()
    # 혼합 정밀도를 사용해 훈련하는 경우 autocast 컨택스트를 사용합니다.
    ctx = torch.autocast(device_type=model.device.type, dtype=model.dtype) \
          if model.dtype in [torch.float16, torch.bfloat16] else nullcontext()
    with ctx:
        generation_output = model.generate(**tokenized_input,
                                           eos_token_id=tokenizer.eos_token_id,
                                           max_new_tokens=max_new_tokens)

    output = tokenizer.batch_decode(generation_output,
                                    skip_special_tokens=skip_special_tokens)
    return output[0]

이제 모델이 실제로 요다체 문장을 생성하는지 테스트해 보죠.

In [23]:
print(generate(model, tokenizer, prompt))

{'input_ids': tensor([[32010,   450, 11004,   338,  4549,   297,   366, 29991, 32007, 32001]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
<|user|> The Force is strong in you!<|end|><|assistant|> Strong in you, the Force is! Yes, hrrrm.<|end|><|endoftext|>


훌륭하네요! 잘 됩니다! 요다처럼, 모델이 말하네요. 흐음..

축하합니다. 첫 번째 LLM을 미세 튜닝했습니다!

Phi-3-mini-4k-instruct 모델에 작은 어댑터를 추가하여 요다 번역기를 만들었습니다! 멋지지 않나요?

### 어댑터 저장하기

훈련이 완료된 후 `Trainer` 객체의 `save_model()` 메서드를 호출해 어댑터를 디스크에 저장할 수 있습니다. 이 메서드는 지정된 폴더에 모든 것을 저장합니다:

In [24]:
trainer.save_model('local-phi3-mini-yoda-adapter')

다음과 같은 파일들이 저장됩니다:

* 어댑터 설정(`adapter_config.json`)과 가중치(`adapter_model.safetensors`). 어댑터 자체 크기는 50MB에 불과합니다.
* 훈련 매개변수값(`training_args.bin`)
* 토크나이저(`tokenizer.json`와 `tokenizer.model`)와 설정(`tokenizer_config.json`), 특수 토큰(`added_tokens.json`와 `speciak_tokens_map.json`)
* README 파일

In [25]:
os.listdir('local-phi3-mini-yoda-adapter')

['adapter_config.json',
 'README.md',
 'training_args.bin',
 'chat_template.jinja',
 'tokenizer.json',
 'tokenizer_config.json',
 'tokenizer.model',
 'added_tokens.json',
 'adapter_model.safetensors',
 'special_tokens_map.json']

이 장에서 만든 어댑터를 다른 사람과 공유하고 싶다면 이를 허깅 페이스 허브에 업로드할 수 있습니다. 먼저 쓰기 권한이 있는 액세스 토큰(access token)을 사용해 허깅 페이스에 로그인합니다:

In [26]:
from huggingface_hub import login
login()

위 코드를 실행하면 다음과 같이 액세스 토큰을 요청합니다:

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch0/hub0.png?raw=True)
<center>Figure 0.1 - Logging into the Hugging Face Hub</center>

A successful login should look like this (pay attention to the permissions):

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch0/hub1.png?raw=True)
<center>Figure 0.2 - Successful Login</center>

그다음 `trainer` 객체의 `push_to_hub()` 메서드를 사용해 자신의 허브 계정에 모두 업로드할 수 있습니다. 모델 이름은 훈련 매개변수 `output_dir` 값에 따라 정해집니다:

In [27]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 27.8kB / 25.2MB            

  ...a-adapter/tokenizer.model:  75%|#######4  |  374kB /  500kB            

  ...adapter/training_args.bin:   2%|2         |   152B / 6.22kB            

CommitInfo(commit_url='https://huggingface.co/haesun/phi3-mini-yoda-adapter/commit/a672452f5aee83d00db47f6c8fd83952e51ad489', commit_message='End of training', commit_description='', oid='a672452f5aee83d00db47f6c8fd83952e51ad489', pr_url=None, repo_url=RepoUrl('https://huggingface.co/haesun/phi3-mini-yoda-adapter', endpoint='https://huggingface.co', repo_type='model', repo_id='haesun/phi3-mini-yoda-adapter'), pr_revision=None, pr_num=None)

이게 끝입니다! 모델이 세상에 공개되었고 누구나 이 모델을 사용해 영어 문장을 요다체로 바꿀 수 있습니다!